In [22]:
# Import Dependencies
import pandas as pd
from sqlalchemy import create_engine

# Convert CSV's into Pandas DataFrames

In [23]:
# Read in csv containing global happiness index data
happy_csv = "../ETL_Project/2020.csv"
happy_df = pd.read_csv(happy_csv)
happy_df.head()

,Country name,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual
0,Finland,Western Europe,7.8087,0.031156,7.869766,7.747634,10.639267,0.954330,71.900825,0.949172,-0.059482,0.195445,1.972317,1.285190,1.499526,0.961271,0.662317,0.159670,0.477857,2.762835
1,Denmark,Western Europe,7.6456,0.033492,7.711245,7.579955,10.774001,0.955991,72.402504,0.951444,0.066202,0.168489,1.972317,1.326949,1.503449,0.979333,0.665040,0.242793,0.495260,2.432741
2,Switzerland,Western Europe,7.5599,0.035014,7.628528,7.491272,10.979933,0.942847,74.102448,0.921337,0.105911,0.303728,1.972317,1.390774,1.472403,1.040533,0.628954,0.269056,0.407946,2.350267
3,Iceland,Western Europe,7.5045,0.059616,7.621347,7.387653,10.772559,0.974670,73.000000,0.948892,0.246944,0.711710,1.972317,1.326502,1.547567,1.000843,0.661981,0.362330,0.144541,2.460688
4,Norway,Western Europe,7.4880,0.034837,7.556281,7.419719,11.087804,0.952487,73.200783,0.955750,0.134533,0.263218,1.972317,1.424207,1.495173,1.008072,0.670201,0.287985,0.434101,2.168266


In [24]:
# Read in csv containing global alcohol consumptioin data
alc_csv = "../ETL_Project/drinksbycountry.csv"
alc_df = pd.read_csv(alc_csv)
alc_df.head()

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
0,Afghanistan,0,0,0,0.0,Asia
1,Albania,89,132,54,4.9,Europe
2,Algeria,25,0,14,0.7,Africa
3,Andorra,245,138,312,12.4,Europe
4,Angola,217,57,45,5.9,Africa


# Clean DataFrames to prepare for SQL Database

In [30]:
# Clean happiness index data
happy_cols = ["Country name",\
              "Ladder score",\
              "Standard error of ladder score",\
              "Logged GDP per capita", "Perceptions of corruption"
             ]
happy_clean = happy_df[happy_cols].copy()

# Rename column headers to match our SQL Database
happy_clean = happy_clean.rename(columns={"Country name":"country",\
                                          "Ladder score":"happiness_index",\
                                          "Standard error of ladder score":"standard_error",\
                                          "Logged GDP per capita":"log_gdp_per_capita",\
                                          "Perceptions of corruption":"perceptions_of_corruption"})

# Remove any duplicate data, round decimal places, and set the index to country
happy_clean = happy_clean.round(2)
happy_clean.drop_duplicates("country", inplace=True)
happy_clean.set_index("country", inplace=True)

happy_clean.head()

,happiness_index,standard_error,log_gdp_per_capita,perceptions_of_corruption
country,,,,
Finland,7.81,0.03,10.64,0.20
Denmark,7.65,0.03,10.77,0.17
Switzerland,7.56,0.04,10.98,0.30
Iceland,7.50,0.06,10.77,0.71
Norway,7.49,0.03,11.09,0.26


In [13]:
# Clean alcohol consumption data
alc_cols = ["country",\
            "continent",\
            "beer_servings",\
            "spirit_servings",\
            "wine_servings",\
            "total_litres_of_pure_alcohol"
           ]
alc_clean = alc_df[alc_cols].copy()

# Rename column headers to match our SQL database
alc_clean = alc_clean.rename(columns={"total_litres_of_pure_alcohol":"litres_pure_alcohol"})

# Remove any duplicate data and set the index to country
alc_clean.drop_duplicates("country", inplace=True)
alc_clean.set_index("country", inplace=True)

alc_clean.head()

,continent,beer_servings,spirit_servings,wine_servings,litres_pure_alcohol
country,,,,,
Afghanistan,Asia,0,0,0,0.0
Albania,Europe,89,132,54,4.9
Algeria,Africa,25,0,14,0.7
Andorra,Europe,245,138,312,12.4
Angola,Africa,217,57,45,5.9


# Create SQL Database Connection

In [14]:
connection = "postgres:sierratahoe@localhost:5432/ETL_Project"
engine = create_engine(f'postgresql://{connection}')

In [15]:
# Confirm our tables
engine.table_names()

['happiness', 'alcohol']

# Load DataFrames into Database

In [16]:
happy_clean.to_sql(name='happiness',con=engine, if_exists='append', index=True)

In [18]:
alc_clean.to_sql(name='alcohol', con=engine, if_exists='append', index=True)